In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
import matplotlib.pylab as pylab
import cartopy.feature as cfeature
from endgame import hist2d
import scipy.stats as stats
import xarray as xr
import ipdb
import glob
import itertools
import numpy.ma as ma
from utils import u_statistics as u_stat
from scipy.stats import gaussian_kde
import pickle as pkl
from utils import u_plot as uplot
import pandas as pd
from scipy.stats import gaussian_kde, linregress
import matplotlib.cm as cm
from utils import u_met
import seaborn
import metpy
from metpy import calc
from metpy.units import units
import statsmodels.api as sm
from scipy import ndimage

import io
import xarray as xr

from numpy.polynomial import polynomial as P

##for regridding, install xesmf:
# conda install esmpy
# pip install xesmf
from utils import constants as cnst

pylab.rcParams['figure.figsize'] = (50., 50.)
%matplotlib inline

In [1]:
cd ..

/home/ck/pythonWorkspace/proj_CEH


In [3]:
var_names = [('u_pl', 'u600', 600), ('u_pl', 'u925', 925), ('u_pl', 'u650', 650) , ('tcwv', 'tcwv', 0)]   #('t_pl', 't925', 925),  , ('t2', 't2', 0)  ### , ('tcwv', 'tcwv', 0)

In [4]:
ftags = ['CP4fut', 'CP4hist'] 
#ftags = ['CP25hist', 'CP25fut']

In [4]:
box = [-5,15,9,20]

In [7]:
import os
for ttime in ftags:
    for vv in var_names:
        print('Doing', ttime, vv[0])
        data_in = xr.open_mfdataset('/media/ck/Elements/Africa/WestAfrica/CP4/'+ttime+'/'+vv[0]+'_daily'+'/'+vv[0]+'_*.nc').sel(longitude=slice(-19,30), latitude=slice(4,30)).squeeze()
        data = data_in[vv[0]]
        if vv[2] != 0:
            data = data.sel(pressure=vv[2]).squeeze()
        if '_pl' in vv[0]:
            data = data.where((data['time.month']>=7) & (data['time.month']<=9) & (data > -1000) & (data != 0) & (data['time.year']>1997)).mean('time').load()  # & (data['time.hour']==12)
        else:
            data = data.where((data['time.month']>=7) & (data['time.month']<=9) & (data['time.year']>1997) ).mean('time').load()  #& (data['time.hour']==12)
        outp = '/home/ck/DIR/cornkle/figs/ENDGAME/dave_map/'+ttime+'_'+vv[1]+'_daily.nc'
        if os.path.isfile(outp):
                os.remove(outp)
        data.to_netcdf(outp)
        del data

Doing CP4fut u_pl


ValueError: Could not find any dimension coordinates to use to order the datasets for concatenation

In [35]:
if 'CP4' in ftags[0]:
    ftag = 'CP4'
else:
    ftag = 'CP25'
    
box = [-5,15,9,20]
    
cp_dic = {}
var_names = ['u', 'tcwv']
for vv in var_names:
    if 'u' in vv:
        hist925 = xr.open_dataarray('/home/ck/DIR/cornkle/figs/ENDGAME/dave_map/'+ftag+'hist_'+vv+'925.nc').sel(longitude=slice(box[0],box[1]), latitude=slice(box[2],box[3])).squeeze()
        hist650 = xr.open_dataarray('/home/ck/DIR/cornkle/figs/ENDGAME/dave_map/'+ftag+'hist_'+vv+'650.nc').sel(longitude=slice(box[0],box[1]), latitude=slice(box[2],box[3])).squeeze()
        hist600 = xr.open_dataarray('/home/ck/DIR/cornkle/figs/ENDGAME/dave_map/'+ftag+'hist_'+vv+'600.nc').sel(longitude=slice(box[0],box[1]), latitude=slice(box[2],box[3])).squeeze()
        
        fut925 = xr.open_dataarray('/home/ck/DIR/cornkle/figs/ENDGAME/dave_map/'+ftag+'fut_'+vv+'925.nc').sel(longitude=slice(box[0],box[1]), latitude=slice(box[2],box[3])).squeeze()
        fut650 = xr.open_dataarray('/home/ck/DIR/cornkle/figs/ENDGAME/dave_map/'+ftag+'fut_'+vv+'650.nc').sel(longitude=slice(box[0],box[1]), latitude=slice(box[2],box[3])).squeeze()
        fut600 = xr.open_dataarray('/home/ck/DIR/cornkle/figs/ENDGAME/dave_map/'+ftag+'fut_'+vv+'600.nc').sel(longitude=slice(box[0],box[1]), latitude=slice(box[2],box[3])).squeeze()
        
        shear_hist = (hist925-hist650)
        shear_fut = (fut925-fut650)
        shear_hist600 = (hist925-hist600)
        shear_fut600 = (fut925-fut600)
        shear_hist600.to_netcdf('/home/ck/DIR/cornkle/figs/ENDGAME/dave_map/'+ftag+'hist_925-600shear.nc')
        shear_fut600.to_netcdf('/home/ck/DIR/cornkle/figs/ENDGAME/dave_map/'+ftag+'fut_925-600shear.nc')
        shear_hist.to_netcdf('/home/ck/DIR/cornkle/figs/ENDGAME/dave_map/'+ftag+'hist_925-650shear.nc')
        shear_fut.to_netcdf('/home/ck/DIR/cornkle/figs/ENDGAME/dave_map/'+ftag+'fut_925-650shear.nc')
        
        cp_dic['shear_hist'] = np.nanmean(shear_hist.values)
        cp_dic['shear_fut'] = np.nanmean(shear_fut.values)
        cp_dic['shear_diff'] = np.nanmean(shear_fut.values) - np.nanmean(shear_hist.values)
        
        cp_dic['shear600_hist'] = np.nanmean(shear_hist600.values)
        cp_dic['shear600_fut'] = np.nanmean(shear_fut600.values)
        cp_dic['shear600_diff'] = np.nanmean(shear_fut600.values) - np.nanmean(shear_hist600.values)
        
    else:
        
        hist = xr.open_dataarray('/home/ck/DIR/cornkle/figs/ENDGAME/dave_map/'+ftag+'hist_'+vv+'.nc')
        fut = xr.open_dataarray('/home/ck/DIR/cornkle/figs/ENDGAME/dave_map/'+ftag+'fut_'+vv+'.nc')
        cp_dic[vv+'_hist'] = np.nanmean(hist.values)
        cp_dic[vv+'_fut'] = np.nanmean(fut.values)
        cp_dic[vv+'_diff'] = np.nanmean(fut.values)-np.nanmean(hist.values)
#ipdb.set_trace()
pd.DataFrame.from_dict(cp_dic, orient='index').to_csv('/home/ck/DIR/cornkle/figs/ENDGAME/dave_map/'+ftag+'_vars.csv')

In [24]:
cc600 = xr.open_mfdataset('/home/ck/DIR/cornkle/figs/ENDGAME/lawrence/daily_shear/cp4_cc_u600_*.nc', concat_dim='dummy', combine='nested').sel(longitude=slice(box[0],box[1]), latitude=slice(box[2],box[3])).load()
cc925 = xr.open_mfdataset('/home/ck/DIR/cornkle/figs/ENDGAME/lawrence/daily_shear/cp4_cc_u925_*.nc', concat_dim='dummy', combine='nested').sel(longitude=slice(box[0],box[1]), latitude=slice(box[2],box[3])).load()
fc600 = xr.open_mfdataset('/home/ck/DIR/cornkle/figs/ENDGAME/lawrence/daily_shear/cp4_fc_u600_*.nc', concat_dim='dummy', combine='nested').sel(longitude=slice(box[0],box[1]), latitude=slice(box[2],box[3])).load()
fc925 = xr.open_mfdataset('/home/ck/DIR/cornkle/figs/ENDGAME/lawrence/daily_shear/cp4_fc_u925_*.nc', concat_dim='dummy', combine='nested').sel(longitude=slice(box[0],box[1]), latitude=slice(box[2],box[3])).load()
cc_shear = (cc925['u925']).mean()-(cc600['u600']).mean()
print(cc_shear)
fc_shear = (fc925['u925']).mean()-(fc600['u600']).mean()
print(fc_shear)
print(fc_shear-cc_shear)

<xarray.DataArray ()>
array(13.315972, dtype=float32)
<xarray.DataArray ()>
array(16.290607, dtype=float32)
<xarray.DataArray ()>
array(2.9746351, dtype=float32)


In [6]:
ftags = ['CP4hist', 'CP4fut', 'CP25hist', 'CP25fut']

path = '/media/ck/Elements/Africa/WestAfrica/CP4/'
for ff in ftags:
    print(ff)
    cp_dic = {}
    cc = xr.open_mfdataset(path+ ff +'/u_pl_daily/*.nc', concat_dim='time', combine='nested').sel(longitude=slice(box[0],box[1]), latitude=slice(box[2],box[3]))
    cc_shear = (cc['u_pl'].sel(pressure=600)-cc['u_pl'].sel(pressure=925)).mean().load()

    cp_dic['shear'] = cc_shear
    del cc
    cc = xr.open_mfdataset(path+ ff + '/tcwv_daily/*.nc', concat_dim='time', combine='nested').sel(longitude=slice(box[0],box[1]), latitude=slice(box[2],box[3]))
    tcwv = cc['tcwv'].mean().load()
    cp_dic['tcwv'] = tcwv
    pd.DataFrame.from_dict(cp_dic, orient='index').to_csv('/home/ck/DIR/cornkle/figs/ENDGAME/dave_map/'+ff+'_vars_DAILY.csv')
    del cc

CP4hist
CP4fut
CP25hist
CP25fut


In [ ]:
cc_shear = (cc925)-(cc600)
print(cc_shear)

Loads bulk statistics derived from having cut out storms (saved in MCSfiles), OBS includes rainfall from TRMM radar

In [27]:

all_files = glob.glob('/media/ck/Elements/Africa/WestAfrica/CP4/CP4hist/lsRain/lsRain_*')
import os
cp4list = {}
years = np.array(np.arange(1998,2007), dtype=str)
months = np.array(['06','07','08','09'])#([ '03', '04', '05', '06', '09', '10', '11'])
days = np.array(np.arange(1,32), dtype=str)

datelist = []
for y,m,d in itertools.product(years, months, days):
    datelist.append(y+m+str(d).zfill(2))


for m in months:
    for y in years:
        #ipdb.set_trace()
        monthfile = []
        ymfiles = [s for s in all_files if str(y)+str(m).zfill(2) in s] 
        #ipdb.set_trace()
        for ymfile in ymfiles:

            try:
                #ipdb.set_trace()
                ll = ymfile #glob.glob(out_path + '_' + str(d) +'00*.nc')[0]
                prcp = xr.open_dataset(ll).sel(longitude=slice(-12,12), latitude=slice(9,18))
                prcp = prcp['lsRain']*3600
            except:
                print('Open error,continue')
                continue
            print('Doing', str(y)+str(m).zfill(2))
            cp4_precip = (prcp).where((prcp > 5))

            monthfile.extend(cp4_precip.values[np.isfinite(cp4_precip.values)])
            #ipdb.set_trace()
        try:
            cp4list[m].append(np.percentile(monthfile,95))
        except:
            cp4list[m] = [np.percentile(monthfile,95)]
        del prcp
        print(m, cp4list[m])

Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
06 [31.73353805541991]
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
06 [31.73353805541991, 28.742917537689213]
Doing 200006
Doing 200006
Doing 200006
Doing 200006
Doing 200006
Doing 200006
Doing 200006
Doing 200006
Doing 200006
Doing 200006
Doing 200006
Doing 20000

Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
07 [36.7343719482422, 31.10868005752563, 37.12237472534178, 33.79096221923828, 31.90028104782104, 38.365998840332026, 35.02125244140625, 36.11232070922851, 36.029558181762695]
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
08 [39.37799758911131]
Doing 199908
Doing 19

Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
09 [35.89686279296875, 33.5196647644043, 36.105804443359375, 34.74776325225827, 35.25209350585938, 36.563804626464844, 37.832775878906226, 35.96463890075684]
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
09 [35.89686279296875, 33.5196647644043, 36.105804443359375, 3

In [28]:

all_files = glob.glob('/media/ck/Elements/Africa/WestAfrica/CP4/CP4fut/lsRain/lsRain_*')
import os
cp4flist = {}
# years = np.array(np.arange(1998,2007), dtype=str)
# months = np.array(['06','07','08','09'])#([ '03', '04', '05', '06', '09', '10', '11'])
# days = np.array(np.arange(1,32), dtype=str)

# datelist = []
# for y,m,d in itertools.product(years, months, days):
#     datelist.append(y+m+str(d).zfill(2))


for m in months:
    for y in years:
        #ipdb.set_trace()
        monthfile = []
        ymfiles = [s for s in all_files if str(y)+str(m).zfill(2) in s] 
        #ipdb.set_trace()
        for ymfile in ymfiles:

            try:
                #ipdb.set_trace()
                ll = ymfile #glob.glob(out_path + '_' + str(d) +'00*.nc')[0]
                prcp = xr.open_dataset(ll).sel(longitude=slice(-12,12), latitude=slice(9,18))
                prcp = prcp['lsRain']*3600
            except:
                print('Open error,continue')
                continue
            print('Doing', str(y)+str(m).zfill(2))
            cp4f_precip = (prcp).where((prcp > 5))

            monthfile.extend(cp4f_precip.values[np.isfinite(cp4f_precip.values)])
            #ipdb.set_trace()
        try:
            cp4flist[m].append(np.percentile(monthfile,95))
        except:
            cp4flist[m] = [np.percentile(monthfile,95)]
        del prcp
        print(m, cp4flist[m])

Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
Doing 199806
06 [44.76148223876955]
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
Doing 199906
06 [44.76148223876955, 40.85256614685056]
Doing 200006
Doing 200006
Doing 200006
Doing 200006
Doing 200006
Doing 200006
Doing 200006
Doing 200006
Doing 200006
Doing 200006
Doing 200006
Doing 200006

Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
Doing 200607
07 [42.96634368896484, 42.531954956054676, 44.72909736633301, 43.13063621520996, 46.65135650634765, 46.63370666503905, 45.20485382080078, 48.1239803314209, 47.76383590698242]
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
Doing 199808
08 [46.972623062133806]
Doing 199908
Doing 19

Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
Doing 200509
09 [48.66824722290039, 49.8304328918457, 50.05772323608397, 45.916491699218746, 44.80621662139884, 49.65371837615966, 46.31918144226074, 47.944309425354]
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
Doing 200609
09 [48.66824722290039, 49.8304328918457, 50.05772323608397, 45.916

In [29]:
for k in cp4list.keys():
    
    print(k, (np.mean(cp4flist[k])-np.mean(cp4list[k])) / np.mean(cp4list[k]) )

06 0.3773814194016784
07 0.2895448348305715
08 0.2812042970639182
09 0.3378371774575678


In [30]:
cp4flist

{'06': [44.76148223876955,
  40.85256614685056,
  37.115652656555156,
  36.91372528076171,
  43.874296188354506,
  47.75864028930664,
  41.0908237457275,
  43.60205383300779,
  49.851177215576165],
 '07': [42.96634368896484,
  42.531954956054676,
  44.72909736633301,
  43.13063621520996,
  46.65135650634765,
  46.63370666503905,
  45.20485382080078,
  48.1239803314209,
  47.76383590698242],
 '08': [46.972623062133806,
  47.17283916473389,
  48.29233856201162,
  44.20218925476074,
  48.174272537231445,
  48.79582386016837,
  47.62482223510733,
  48.73942565917961,
  49.71803131103515],
 '09': [48.66824722290039,
  49.8304328918457,
  50.05772323608397,
  45.916491699218746,
  44.80621662139884,
  49.65371837615966,
  46.31918144226074,
  47.944309425354,
  47.004756927490206]}

In [31]:
cp4list

{'06': [31.73353805541991,
  28.742917537689213,
  31.731615447998024,
  28.125846481323237,
  30.309811782836892,
  34.089009094238264,
  29.194100379943812,
  34.03018283843987,
  32.15450897216797],
 '07': [36.7343719482422,
  31.10868005752563,
  37.12237472534178,
  33.79096221923828,
  31.90028104782104,
  38.365998840332026,
  35.02125244140625,
  36.11232070922851,
  36.029558181762695],
 '08': [39.37799758911131,
  34.655555725097656,
  38.41624221801756,
  37.01994705200197,
  35.07526969909665,
  36.8545471191406,
  39.25683803558352,
  36.70153198242187,
  38.02368507385253],
 '09': [35.89686279296875,
  33.5196647644043,
  36.105804443359375,
  34.74776325225827,
  35.25209350585938,
  36.563804626464844,
  37.832775878906226,
  35.96463890075684,
  35.681192588806155]}

In [ ]:
all_files = glob.glob('/media/ck/Elements/Africa/WestAfrica/CP4/CP4hist/lsRain/lsRain_*')
import os
cp4list = {'06' : [], '07' : [], '08' : [], '09' : []}
years = np.array(np.arange(1998,2007), dtype=str)
months = np.array(['06','07','08','09'])#([ '03', '04', '05', '06', '09', '10', '11'])
days = np.array(np.arange(1,32), dtype=str)

datelist = []

for y,m,d in itertools.product(years, months, days):
    datelist.append(y+m+str(d).zfill(2))


for m in months:
    for y in years:
        #ipdb.set_trace()
        monthfile = []
        ymfiles = [s for s in all_files if str(y)+str(m).zfill(2) in s] 
        #ipdb.set_trace()
        for ymfile in ymfiles:

            try:
                #ipdb.set_trace()
                ll = ymfile #glob.glob(out_path + '_' + str(d) +'00*.nc')[0]
                prcp = xr.open_dataset(ll, decode_time=False).sel(longitude=slice(-17,15), latitude=slice(4.5,20))
                prcp = prcp['lsRain']*3600
                prcp = prcp.resample(time="1D").sum()
                prcp.values[prcp.values<1]=np.nan
            except:
                print('Open error,continue')
                continue
            print('Doing', str(y)+str(m).zfill(2))
            try:
                cp4list[m] = xr.concat([cp4list[m],prcp], dim='dummy')
                ipdb.set_trace()
            except:
                cp4list[m] = prcp
            #ipdb.set_trace()

Doing 199806
Doing 199806
> <ipython-input-5-364c92ac8bfc>(20)<module>()
     19         #ipdb.set_trace()
---> 20         for ymfile in ymfiles:
     21 

ipdb> cp4list['06']
<xarray.DataArray 'lsRain' (dummy: 2, time: 2, latitude: 383, longitude: 790)>
array([[[[23.954031 , 23.062752 , 21.271257 , ..., 15.953993 ,
          20.599981 , 28.44997  ],
         [18.687874 , 15.616014 ,  6.3602324, ..., 16.341215 ,
          19.313868 , 25.7206   ],
         [14.382782 , 10.885391 ,  4.6426287, ..., 20.657408 ,
          22.947493 , 27.096481 ],
         ...,
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan]],

        [[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        na